<a href="https://colab.research.google.com/github/aethelind/notebooks-misc/blob/main/unique_solution_synthetic_aaai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Decision-Focused Learning Simple Synthetic Unique Problem 1**

# Get required files

In [2]:
# Clear out directory
!rm -rf *
# Download data_decisions_benchmarks.zip and unzip diverse_recommendation_data.pickle
!curl https://bryanwilder.github.io/files/data_decisions_benchmarks.zip | jar xv benchmarks_release/diverse_recommendation_data.pickle
# Move diverse_recommendation_data.pickle to current directory
!mv benchmarks_release/diverse_recommendation_data.pickle .
# Remove empty directory
!rm -rf benchmarks_release
# Download hetrec2011-movielens-2k-v2.zip and unzip movie_actors.dat and user_ratedmovies.dat
!curl https://files.grouplens.org/datasets/hetrec2011/hetrec2011-movielens-2k-v2.zip | jar xv movie_actors.dat user_ratedmovies.dat

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 83.0M  100 83.0M    0     0  11.6M      0  0:00:07  0:00:07 --:--:-- 11.0M
 inflated: benchmarks_release/diverse_recommendation_data.pickle
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0 17.9M    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0 inflated: movie_actors.dat
 54 17.9M   54  9.8M    0     0  7949k      0  0:00:02  0:00:01  0:00:01 7943k inflated: user_ratedmovies.dat
100 17.9M  100 17.9M    0     0  10.1M      0  0:00:01  0:00:01 --:--:-- 10.1M


# Libraries

In [3]:
## submodular.py
import torch

class ContinuousOptimizer(torch.autograd.Function):
    """
    pytorch module for differentiable submodular maximization. The forward pass 
    computes the optimal x for given parameters. The backward pass differentiates 
    that optimal x wrt the parameters.
    """

    @staticmethod
    def forward(ctx, params, optimize_func, get_dgradf_dparams, get_hessian=None, max_x=1., verbose=False):
        """
        Computes the optimal x using the supplied optimizer. 
        """
        ctx.optimize_func = optimize_func
        ctx.get_dgradf_dparams = get_dgradf_dparams
        ctx.verbose = verbose
        ctx.get_hessian = get_hessian
        ctx.all_xs = []
        ctx.max_x = max_x

        import numpy as np
        with torch.enable_grad():
            x = ctx.optimize_func(params, verbose=ctx.verbose)
        ctx.x = x.data
        ctx.all_xs.append(ctx.x.detach().numpy())
        ctx.params = params
        ctx.xgrad = x.grad.data
        return x.data

    @staticmethod
    def backward(ctx, grad_output):
        """
        Differentiates the optimal x returned by the forward pass with respect
        to the ratings matrix that was given as input.
        """
        import numpy as np
        from torch.autograd import Variable
        x = ctx.x
        params = ctx.params
        xgrad = ctx.xgrad
        dxdr = ContinuousOptimizer.get_dxdr(x.detach().numpy(), -xgrad.detach().numpy(), params.detach(
        ).numpy(), ctx.get_dgradf_dparams, ctx.get_hessian, ctx.max_x)
        dxdr_t = torch.from_numpy(np.transpose(dxdr))
        out = torch.mm(dxdr_t.float(), grad_output.view(len(x), 1))
        return out.view_as(params), None, None, None, None

    @staticmethod
    def get_dxdr(x, grad, params, get_dgradf_dparams, get_hessian, max_x):
        '''
        Returns the derivative of the optimal solution in the region around x in 
        terms of the rating matrix r. 

        x: an optimal solution

        grad: df/dx at x

        params: the current parameter settings
        '''
        import numpy as np
        import scipy as sp
        import scipy.sparse
        import scipy.linalg
        n = len(x)
        # first get the optimal dual variables via the KKT conditions
        # dual variable for constraint sum(x) <= k
        if np.logical_and(x > 0, x < max_x).any():
            lambda_sum = np.mean(grad[np.logical_and(x > 0, x < max_x)])
        else:
            lambda_sum = 0
        # dual variable for constraint x <= max_x
        lambda_upper = []
        # dual variable for constraint x >= 0
        lambda_lower = []
        for i in range(n):
            if np.abs(x[i] - max_x) < 0.000001:
                lambda_upper.append(grad[i] - lambda_sum)
            else:
                lambda_upper.append(0)
            if x[i] > 0:
                lambda_lower.append(0)
            else:
                lambda_lower.append(grad[i] - lambda_sum)
        # number of constraints
        m = 2*n + 1
        # collect value of dual variables
        lam = np.zeros((m))
        lam[0] = lambda_sum
        lam[1:(n+1)] = lambda_upper
        lam[n+1:] = lambda_lower
        diag_lambda = np.matrix(np.diag(lam))
        # collect value of constraints
        g = np.zeros((m))
        # TODO: replace the second x.sum() with k so that this is actually generally correct
        g[0] = x.sum() - x.sum()
        g[1:(n+1)] = x - max_x
        g[n+1:] = -x
        diag_g = np.matrix(np.diag(g))
        # gradient of constraints wrt x
        dgdx = np.zeros((m, n))
        # gradient of constraint sum(x) <= k
        dgdx[0, :] = 1
        # gradient of constraints x <= 1
        for i in range(1, n+1):
            dgdx[i, i-1] = 1
        # gradient of constraints x >= 0 <--> -x <= 0
        for i in range(n+1, m):
            dgdx[i, i-(n+1)] = -1
        dgdx = np.matrix(dgdx)
        # the Hessian matrix -- all zeros for now
        if get_hessian == None:
            H = np.matrix(np.zeros((n, n)))
        else:
            H = get_hessian(x, params)
        # coefficient matrix for the linear system
        A = np.bmat([[H, np.transpose(dgdx)], [diag_lambda*dgdx, diag_g]])
        # add 0.01*I to improve conditioning
        A = A + 0.01*np.eye(n+m)
        # RHS of the linear system, mostly partial derivative of grad f wrt params
        dgradf_dparams = get_dgradf_dparams(x, params, num_samples=1000)
        reshaped = np.zeros(
            (dgradf_dparams.shape[0], dgradf_dparams.shape[1]*dgradf_dparams.shape[2]))
        for i in range(n):
            reshaped[i] = dgradf_dparams[i].flatten()
        b = np.bmat([[reshaped], [np.zeros((m, reshaped.shape[1]))]])
        # solution to the system
        derivatives = sp.linalg.solve(A, b)
        if np.isnan(derivatives).any():
            print('report')
            print(np.isnan(A).any())
            print(np.isnan(b).any())
            print(np.isnan(dgdx).any())
            print(np.isnan(diag_lambda).any())
            print(np.isnan(diag_g).any())
            print(np.isnan(dgradf_dparams).any())
        # first n are derivatives of primal variables
        derivatives = derivatives[:n]
        return derivatives


In [4]:
# coverage.py
import torch
import numpy as np
from numba import jit


@jit
def gradient_coverage(x, P, w):
    n = len(w)
    m = len(x)
    grad = np.zeros(m, dtype=np.float32)
    for i in range(n):
        p_fail = 1 - x*P[:, i]
        p_all_fail = np.prod(p_fail)
        for j in range(m):
            grad[j] += w[i] * P[j, i] * p_all_fail/p_fail[j]
    return grad


@jit
def hessian_coverage(x, P, w):
    n = len(w)
    m = len(x)
    hessian = np.zeros((m, m), dtype=np.float32)
    for i in range(n):
        p_fail = 1 - x*P[:, i]
        p_all_fail = np.prod(p_fail)
        for j in range(m):
            for k in range(m):
                hessian[j, k] = -w[i] * P[j, i] * \
                    p_all_fail/(p_fail[j] * p_fail[k])
    return hessian


@jit
def objective_coverage(x, P, w):
    n = len(w)
    total = 0
    for i in range(n):
        p_fail = 1 - x*P[:, i]
        p_all_fail = np.prod(p_fail)
        total += w[i] * (1 - p_all_fail)
    return total


class CoverageInstanceMultilinear(torch.autograd.Function):
    """
    Represents a coverage instance with given coverage probabilities
    P and weights w. Forward pass computes the objective value (if evaluate_forward
    is true). Backward computes the gradients wrt decision variables x.
    """
    @staticmethod
    def forward(ctx, x, P, w, evaluate_forward):
        ctx.evaluate_forward = evaluate_forward
        if type(P) != np.ndarray:
            P = P.detach().numpy()
        if type(w) != np.ndarray:
            w = w.detach().numpy()
        ctx.P = P
        ctx.w = w

        ctx.x = x.detach().numpy()
        if ctx.evaluate_forward:
            out = objective_coverage(ctx.x, ctx.P, ctx.w)
        else:
            out = -1
        return torch.tensor(out).float()

    @staticmethod
    def backward(ctx, grad_in):
        grad = gradient_coverage(ctx.x, ctx.P, ctx.w)
        return torch.from_numpy(grad).float()*grad_in.float(), None, None, None


def optimize_coverage_multilinear(P, w, verbose=True, k=10, c=1., minibatch_size=None):
    '''
    Run some variant of SGD for the coverage problem with given 
    coverage probabilities P and weights w

    '''
    import torch
    # from utils import project_uniform_matroid_boundary as project

    # objective which will provide gradient evaluations
    # coverage = CoverageInstanceMultilinear.apply(P, w, verbose) # move to call below
    # decision variables
    x = torch.zeros(P.shape[0], requires_grad=True)
    # set up the optimizer
    learning_rate = 0.1
    optimizer = torch.optim.SGD(
        [x], momentum=0.9, lr=learning_rate, nesterov=True)
    # take projected stochastic gradient steps
    for t in range(10):
        loss = -CoverageInstanceMultilinear.apply(x, P, w, verbose)
        if verbose:
            print(t, -loss.item())
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()
        x.data = torch.from_numpy(project_uniform_matroid_boundary(x.data.numpy(), k, 1/c)).float()
    return x


@jit
def dgrad_coverage(x, P, num_samples, w):
    n = len(w)
    m = len(x)
    dgrad = np.zeros((m, m, n), dtype=np.float32)
    for i in range(n):
        p_fail = 1 - x*P[:, i]
        p_all_fail = np.prod(p_fail)
        for j in range(m):
            for k in range(m):
                if j == k:
                    dgrad[j, k, i] = w[i] * p_all_fail/p_fail[j]
                else:
                    dgrad[j, k, i] = -w[i] * x[k] * P[j, i] * \
                        p_all_fail/(p_fail[j] * p_fail[k])
    return dgrad


@jit
def dgrad_coverage_stochastic(x, P, num_samples, w, num_real_samples):
    n = len(w)
    m = len(x)
    rand_rows = np.random.choice(list(range(m)), num_real_samples)
    rand_cols = np.random.choice(list(range(n)), num_real_samples)

    dgrad = np.zeros((m, m, n), dtype=np.float32)

    p_fail = np.zeros((n, m), dtype=np.float32)
    p_all_fail = np.zeros((n), dtype=np.float32)
    for i in range(n):
        p_fail[i] = 1 - x*P[:, i]
        p_all_fail[i] = np.prod(p_fail[i])

    for sample in range(num_real_samples):
        k = rand_rows[sample]
        i = rand_cols[sample]
        for j in range(m):
            if j == k:
                dgrad[j, k, i] = w[i] * p_all_fail[i]/p_fail[i, j]
            else:
                dgrad[j, k, i] = -w[i] * x[k] * P[j, i] * \
                    p_all_fail[i]/(p_fail[i, j] * p_fail[i, k])
    return dgrad


In [5]:
## utils.py

def project_uniform_matroid_boundary(x, k, c=1):
    '''
    Exact projection algorithm of Karimi et al. This is the projection implementation
    that should be used now.
    
    Projects x onto the set {y: 0 <= y <= 1/c, ||y||_1 = k}
    '''
    import numpy as np
    k *= c
    n = len(x)
    x = x.copy()
    alpha_upper = x/c
    alpha_lower = (x*c - 1)/c**2
    S = []
    S.extend(alpha_lower)
    S.extend(alpha_upper)
    S.sort()
    S = np.unique(S)
    h = n
    alpha = min(S) - 1
    m = 0
    for i in range(len(S)):
        hprime = h + (S[i] - alpha)*m
        if hprime < k and k <= h:
            alphastar = (S[i] - alpha)*(h - k)/(h - hprime) + alpha
            result = np.zeros((n))
            for j in range(n):
                if alpha_lower[j] > alphastar:
                    result[j] = 1./c
                elif alpha_upper[j] >= alphastar:
                    result[j] = x[j] - alphastar*c
            return result
        m -= (alpha_lower == S[i]).sum()*(c**2)
        m += (alpha_upper == S[i]).sum()*(c**2)
        h = hprime
        alpha = S[i]
    raise Exception('projection did not terminate')

def project_cvx(x, k):
    '''
    Exact Euclidean projection onto the boundary of the k uniform matroid polytope.
    '''
    from cvxpy import Variable, Minimize, sum_squares, Problem
    import numpy as np
    n = len(x)
    p = Variable(n, 1)
    objective = Minimize(sum_squares(p - x))
    constraints = [sum(p) == k, p >= 0, p <= 1]
    prob = Problem(objective, constraints)
    prob.solve()
    return np.reshape(np.array(p.value), x.shape)


### Load & Preprocess Data



The probabilities of coverage below has none of the targets match.

Thus, it becomes impossible to cover all 4 targets without at least k=4. The highest objective value we can obtain becomes k.


In [44]:
# load probability matrix 
P_list = [
[	0	,	0	,	0	,	1	],
[	0	,	0	,	0	,	0	],
[	1	,	1	,	0	,	0	],
[	0	,	0	,	0	,	0	],
[	0	,	0	,	0	,	0	],
[	0	,	0	,	0	,	0	],
[	0	,	0	,	0	,	0	],
[	0	,	0	,	0	,	0	],
[	0	,	0	,	0	,	1	],
[	0	,	0	,	0	,	0	],
[	0	,	0	,	0	,	0	],
[	1	,	0	,	0	,	1	],
[	0	,	0	,	0	,	0	],
[	1	,	0	,	0	,	0	],
[	1	,	0	,	0	,	0	],
[	0	,	0	,	0	,	1	],
[	0	,	0	,	0	,	1	],
[	0	,	0	,	0	,	1	],
[	0	,	0	,	0	,	1	],
[	0	,	0	,	0	,	0	],
[	0	,	0	,	0	,	0	],
[	0	,	0	,	0	,	1	],
[	0	,	0	,	0	,	1	],
[	1	,	0	,	0	,	1	],
[	0	,	1	,	1	,	0	],
[	0	,	0	,	0	,	1	],
]



In [45]:
# load features
circuit_km = [3.7, 
3, 
6.9, 
3.5, 
2, 
2.8, 
2.4, 
4.5, 
4.3, 
2.6, 
3.2, 
4.9, 
1.9, 
4.1, 
3.4, 
4.3, 
4, 
3.3, 
3.4, 
2.4, 
1.9, 
4.5, 
4, 
4.9, 
1.2, 
3.9, 
]
y = []
for i in circuit_km:
  y.append([i])

In [58]:
## recommendation_nn_decision.py
import numpy as np
import torch
# from coverage import optimize_coverage_multilinear, CoverageInstanceMultilinear, dgrad_coverage, hessian_coverage
import pickle
from functools import partial
# from submodular import ContinuousOptimizer
import torch.nn as nn
import random
# import argparse

# parser = argparse.ArgumentParser()
# parser.add_argument('--layers', type=int, default=1)
# parser.add_argument('--activation', type=str, default='relu')
# parser.add_argument('--k', type=int, default=20)

# args = parser.parse_args()
num_layers = 4
activation = 'relu'
k = 2
use_hessian = False
num_iters = 500
instance_sizes = [0]
learning_rate = 1e-4

Ps = {}
data = {}
f_true = {}

for num_items in instance_sizes:
    Ps_size = np.array(P_list)
    data_size = np.array(y)

    num_targets = Ps_size.shape[1] #500 --> 4
    num_features = data_size.shape[1] #2113 --> 1
    Ps[num_items] = [torch.from_numpy(Ps_size).long()]
    data[num_items] = [torch.from_numpy(data_size).float()]
    w = np.ones(num_targets, dtype=np.float32)
    f_true[num_items] = [(P, w) for P in Ps[num_items]]
  
num_repetitions = 0

train = {}
test = {}
for size in instance_sizes:
  train[size], test[size] = np.array(P_list), np.array(y)

### Train NN

In [59]:
vals = np.zeros((num_repetitions+1, len(instance_sizes), len(instance_sizes)))

for idx in range(num_repetitions, num_repetitions + 1):

    intermediate_size = 26

    def make_fc():
        if num_layers > 1:
            if activation == 'relu':
                activation_fn = nn.ReLU
            elif activation == 'sigmoid':
                activation_fn = nn.Sigmoid
            else:
                raise Exception(
                    'Invalid activation function: ' + str(activation))
            net_layers = [
                nn.Linear(num_features, intermediate_size), activation_fn()]
            for hidden in range(num_layers-2):
                net_layers.append(
                    nn.Linear(intermediate_size, intermediate_size))
                net_layers.append(activation_fn())
            net_layers.append(nn.Linear(intermediate_size, num_targets))
            net_layers.append(nn.Sigmoid())
            return nn.Sequential(*net_layers)
        else:
            return nn.Sequential(nn.Linear(num_features, num_targets), nn.Sigmoid())

    # optimizer that will be used for training (and testing)
    optfunc = partial(optimize_coverage_multilinear, w=w, k=k, c=0.95)
    dgrad = partial(dgrad_coverage, w=w)
    if use_hessian:
        hessian = partial(hessian_coverage, w=w)
    else:
        hessian = None

    # runs the given net on instances of a given size
    def eval_opt(net, instances, size):
        net.eval()
        val = 0.
        for i in range(len(instances)):
            pred = net(data[size][i])
            x = ContinuousOptimizer.apply(pred, optfunc, dgrad, None, 0.95)
            pp, ww = f_true[size][i]
            val += objective_coverage(x.detach().numpy(), pp.detach().numpy(), ww)
        net.train()
        return val/len(instances), pred, x

    # train a network for each size, and test on each sizes
    for train_idx, train_size in enumerate(instance_sizes):
        net = make_fc()
        optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
        # training
        for t in range(num_iters):
            print(f"Iteration {t}")
            #i = random.randint(0, 80)
            i=0
            pred = net(data[train_size][i])
            x = ContinuousOptimizer.apply(pred, optfunc, dgrad, None, 0.95)
            pp, ww = f_true[train_size][i]
            loss = -CoverageInstanceMultilinear.apply(x, pp, ww, False)
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()
        # save learned network state
        savepath = '/tmp/net_diffopt_smalllr_{0}_{1}_{2}_{3}.pt'.format(
            train_size, k, num_layers, idx)
        torch.save(net.state_dict(), savepath)
        # test on different sizes
        for test_idx, test_size in enumerate(instance_sizes):
            vals[idx, train_idx, test_idx], return_prediction, return_x = eval_opt(net, test, test_size)
            print(vals[idx, train_idx, test_idx])
        # save out values
        print(idx, train_size, vals[idx, train_idx])
        with open('results_recommendation_' + str(num_layers) + '.pickle', 'wb') as f:
            pickle.dump(vals, f)


Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Iteration 41
Iteration 42
Iteration 43
Iteration 44
Iteration 45
Iteration 46
Iteration 47
Iteration 48
Iteration 49
Iteration 50
Iteration 51
Iteration 52
Iteration 53
Iteration 54
Iteration 55
Iteration 56
Iteration 57
Iteration 58
Iteration 59
Iteration 60
Iteration 61
Iteration 62
Iteration 63
Iteration 64
Iteration 65
Iteration 66
Iteration 67
Iteration 68
Iteration 69
Iteration 70
Iteration 71
Iteration 72
Iteration 73
Iteration 74
Iteration 75
Iteration 76
Iteration

In [60]:
data[train_size][0]

tensor([[3.7000],
        [3.0000],
        [6.9000],
        [3.5000],
        [2.0000],
        [2.8000],
        [2.4000],
        [4.5000],
        [4.3000],
        [2.6000],
        [3.2000],
        [4.9000],
        [1.9000],
        [4.1000],
        [3.4000],
        [4.3000],
        [4.0000],
        [3.3000],
        [3.4000],
        [2.4000],
        [1.9000],
        [4.5000],
        [4.0000],
        [4.9000],
        [1.2000],
        [3.9000]])

In [61]:
return_prediction

tensor([[0.6161, 0.5634, 0.6617, 0.4840],
        [0.6082, 0.5444, 0.6413, 0.4779],
        [0.6509, 0.6444, 0.7491, 0.5106],
        [0.6139, 0.5580, 0.6559, 0.4822],
        [0.5973, 0.5183, 0.6130, 0.4706],
        [0.6060, 0.5391, 0.6355, 0.4764],
        [0.6016, 0.5287, 0.6243, 0.4735],
        [0.6251, 0.5850, 0.6844, 0.4909],
        [0.6228, 0.5796, 0.6788, 0.4892],
        [0.6038, 0.5338, 0.6299, 0.4749],
        [0.6105, 0.5499, 0.6472, 0.4796],
        [0.6298, 0.5960, 0.6955, 0.4943],
        [0.5961, 0.5155, 0.6101, 0.4699],
        [0.6206, 0.5742, 0.6732, 0.4874],
        [0.6127, 0.5553, 0.6530, 0.4814],
        [0.6228, 0.5796, 0.6788, 0.4892],
        [0.6195, 0.5716, 0.6703, 0.4866],
        [0.6116, 0.5526, 0.6501, 0.4805],
        [0.6127, 0.5553, 0.6530, 0.4814],
        [0.6016, 0.5287, 0.6243, 0.4735],
        [0.5961, 0.5155, 0.6101, 0.4699],
        [0.6251, 0.5850, 0.6844, 0.4909],
        [0.6195, 0.5716, 0.6703, 0.4866],
        [0.6298, 0.5960, 0.6955, 0

In [62]:
pred = net(data[train_size][0])
pred

tensor([[0.6161, 0.5634, 0.6617, 0.4840],
        [0.6082, 0.5444, 0.6413, 0.4779],
        [0.6509, 0.6444, 0.7491, 0.5106],
        [0.6139, 0.5580, 0.6559, 0.4822],
        [0.5973, 0.5183, 0.6130, 0.4706],
        [0.6060, 0.5391, 0.6355, 0.4764],
        [0.6016, 0.5287, 0.6243, 0.4735],
        [0.6251, 0.5850, 0.6844, 0.4909],
        [0.6228, 0.5796, 0.6788, 0.4892],
        [0.6038, 0.5338, 0.6299, 0.4749],
        [0.6105, 0.5499, 0.6472, 0.4796],
        [0.6298, 0.5960, 0.6955, 0.4943],
        [0.5961, 0.5155, 0.6101, 0.4699],
        [0.6206, 0.5742, 0.6732, 0.4874],
        [0.6127, 0.5553, 0.6530, 0.4814],
        [0.6228, 0.5796, 0.6788, 0.4892],
        [0.6195, 0.5716, 0.6703, 0.4866],
        [0.6116, 0.5526, 0.6501, 0.4805],
        [0.6127, 0.5553, 0.6530, 0.4814],
        [0.6016, 0.5287, 0.6243, 0.4735],
        [0.5961, 0.5155, 0.6101, 0.4699],
        [0.6251, 0.5850, 0.6844, 0.4909],
        [0.6195, 0.5716, 0.6703, 0.4866],
        [0.6298, 0.5960, 0.6955, 0

In [64]:
f_true[0][0][0]

tensor([[0, 0, 0, 1],
        [0, 0, 0, 0],
        [1, 1, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [1, 0, 0, 1],
        [0, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [1, 0, 0, 1],
        [0, 1, 1, 0],
        [0, 0, 0, 1]])

In [65]:
f_true[0][0][0]-pred

tensor([[-0.6161, -0.5634, -0.6617,  0.5160],
        [-0.6082, -0.5444, -0.6413, -0.4779],
        [ 0.3491,  0.3556, -0.7491, -0.5106],
        [-0.6139, -0.5580, -0.6559, -0.4822],
        [-0.5973, -0.5183, -0.6130, -0.4706],
        [-0.6060, -0.5391, -0.6355, -0.4764],
        [-0.6016, -0.5287, -0.6243, -0.4735],
        [-0.6251, -0.5850, -0.6844, -0.4909],
        [-0.6228, -0.5796, -0.6788,  0.5108],
        [-0.6038, -0.5338, -0.6299, -0.4749],
        [-0.6105, -0.5499, -0.6472, -0.4796],
        [ 0.3702, -0.5960, -0.6955,  0.5057],
        [-0.5961, -0.5155, -0.6101, -0.4699],
        [ 0.3794, -0.5742, -0.6732, -0.4874],
        [ 0.3873, -0.5553, -0.6530, -0.4814],
        [-0.6228, -0.5796, -0.6788,  0.5108],
        [-0.6195, -0.5716, -0.6703,  0.5134],
        [-0.6116, -0.5526, -0.6501,  0.5195],
        [-0.6127, -0.5553, -0.6530,  0.5186],
        [-0.6016, -0.5287, -0.6243, -0.4735],
        [-0.5961, -0.5155, -0.6101, -0.4699],
        [-0.6251, -0.5850, -0.6844

In [66]:
pred.round()

tensor([[1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 0., 1., 0.],
        [1., 1., 1., 0.]], grad_fn=<RoundBackward0>)

In [67]:
return_x

tensor([0.0000, 0.0000, 0.9500, 0.0000, 0.0000, 0.0000, 0.0000, 0.1456, 0.0889,
        0.0000, 0.0000, 0.2661, 0.0000, 0.0341, 0.0000, 0.0889, 0.0073, 0.0000,
        0.0000, 0.0000, 0.0000, 0.1456, 0.0073, 0.2661, 0.0000, 0.0000],
       grad_fn=<ContinuousOptimizerBackward>)

In [68]:
return_x.round()

tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.], grad_fn=<RoundBackward0>)

In [69]:
pp, ww = f_true[size][i]

In [70]:
x = ContinuousOptimizer.apply(pred, optfunc, dgrad, None, 0.95)

In [71]:
x

tensor([0.0000, 0.0000, 0.9500, 0.0000, 0.0000, 0.0000, 0.0000, 0.1456, 0.0889,
        0.0000, 0.0000, 0.2661, 0.0000, 0.0341, 0.0000, 0.0889, 0.0073, 0.0000,
        0.0000, 0.0000, 0.0000, 0.1456, 0.0073, 0.2661, 0.0000, 0.0000],
       grad_fn=<ContinuousOptimizerBackward>)

In [72]:
n = len(ww)
total = 0
for i in range(n):
  p_fail = 1 - return_x.detach().numpy()*return_prediction.detach().numpy()[:, i]
  p_all_fail = np.prod(p_fail)
  total += ww[i] * (1 - p_all_fail)

In [73]:
total

3.178190588951111

In [74]:
objective_coverage(x.detach().numpy(), pp.detach().numpy(), ww)

2.5475668273866177

In [75]:
x.topk(2)

torch.return_types.topk(
values=tensor([0.9500, 0.2661], grad_fn=<TopkBackward0>),
indices=tensor([ 2, 23]))

In [77]:
[f_true[0][0][0][2], f_true[0][0][0][23]]

[tensor([1, 1, 0, 0]), tensor([1, 0, 0, 1])]

With this problem I would get a group of 3 customers out of a possible optimal 4 customers. This is the same as the two-stage approach. 

We still haven't coerced the algo to giving an infeasible solution because the cardinality constraint is imposed at the end of the algo.

## Two-Stage Approach

In [46]:
vals = np.zeros((num_repetitions+1, len(instance_sizes), len(instance_sizes)))

for idx in range(num_repetitions, num_repetitions + 1):

    intermediate_size = 26

    def make_fc():
        if num_layers > 1:
            if activation == 'relu':
                activation_fn = nn.ReLU
            elif activation == 'sigmoid':
                activation_fn = nn.Sigmoid
            else:
                raise Exception(
                    'Invalid activation function: ' + str(activation))
            net_layers = [
                nn.Linear(num_features, intermediate_size), activation_fn()]
            for hidden in range(num_layers-2):
                net_layers.append(
                    nn.Linear(intermediate_size, intermediate_size))
                net_layers.append(activation_fn())
            net_layers.append(nn.Linear(intermediate_size, num_targets))
            net_layers.append(nn.Sigmoid())
            return nn.Sequential(*net_layers)
        else:
            return nn.Sequential(nn.Linear(num_features, num_targets), nn.Sigmoid())

    # optimizer that will be used for training (and testing)
    optfunc = partial(optimize_coverage_multilinear, w=w, k=k, c=0.95)
    dgrad = partial(dgrad_coverage, w=w)
    if use_hessian:
        hessian = partial(hessian_coverage, w=w)
    else:
        hessian = None

    # runs the given net on instances of a given size
    def eval_opt(net, instances, size):
        net.eval()
        val = 0.
        for i in range(len(instances)):
            pred = net(data[size][i])
            x = ContinuousOptimizer.apply(pred, optfunc, dgrad, None, 0.95)
            pp, ww = f_true[size][i]
            val += objective_coverage(x.detach().numpy(), pp.detach().numpy(), ww)
        net.train()
        return val/len(instances), pred, x
        
    def eval_loss(net, instances, size):
        net.eval()
        val = 0.
        for i in range(len(instances)):
            pred = net(data[size][i])
            pp, ww = f_true[size][i]
            val += loss_fn(pred, pp)
        net.train()
        return val/len(instances), pred, x

    # train a network for each size, and test on each sizes
    for train_idx, train_size in enumerate(instance_sizes):
        net_two_stage = make_fc()
        loss_fn = nn.MultiLabelSoftMarginLoss()
        optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
        # training
        for t in range(num_iters):
            print(f"Iteration {t}")
            i=0
            pred = net_two_stage(data[train_size][i])
            pp, ww = f_true[train_size][i]
            loss = loss_fn(pred, pp)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # save learned network state
        savepath = '/tmp/net_two_stage_diffopt_smalllr_{0}_{1}_{2}_{3}.pt'.format(
            train_size, k, num_layers, idx)
        torch.save(net_two_stage.state_dict(), savepath)
        # test on different sizes
        for test_idx, test_size in enumerate(instance_sizes):
            vals[idx, train_idx, test_idx], prediction_two_stage, x_two_stage = eval_opt(net_two_stage, test, test_size)
            print(vals[idx, train_idx, test_idx])
        # save out values
        print(idx, train_size, vals[idx, train_idx])
        with open('results_recommendation_' + str(num_layers) + '.pickle', 'wb') as f:
            pickle.dump(vals, f)


Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Iteration 41
Iteration 42
Iteration 43
Iteration 44
Iteration 45
Iteration 46
Iteration 47
Iteration 48
Iteration 49
Iteration 50
Iteration 51
Iteration 52
Iteration 53
Iteration 54
Iteration 55
Iteration 56
Iteration 57
Iteration 58
Iteration 59
Iteration 60
Iteration 61
Iteration 62
Iteration 63
Iteration 64
Iteration 65
Iteration 66
Iteration 67
Iteration 68
Iteration 69
Iteration 70
Iteration 71
Iteration 72
Iteration 73
Iteration 74
Iteration 75
Iteration 76
Iteration

In [47]:
prediction_two_stage

tensor([[0.5028, 0.5914, 0.5565, 0.4370],
        [0.5095, 0.5842, 0.5555, 0.4463],
        [0.4734, 0.6227, 0.5620, 0.3868],
        [0.5047, 0.5894, 0.5562, 0.4397],
        [0.5190, 0.5738, 0.5520, 0.4577],
        [0.5114, 0.5821, 0.5548, 0.4486],
        [0.5152, 0.5780, 0.5534, 0.4531],
        [0.4954, 0.5994, 0.5574, 0.4251],
        [0.4972, 0.5974, 0.5573, 0.4283],
        [0.5133, 0.5801, 0.5541, 0.4509],
        [0.5076, 0.5863, 0.5557, 0.4438],
        [0.4916, 0.6034, 0.5576, 0.4186],
        [0.5200, 0.5728, 0.5517, 0.4588],
        [0.4991, 0.5954, 0.5572, 0.4315],
        [0.5056, 0.5884, 0.5560, 0.4411],
        [0.4972, 0.5974, 0.5573, 0.4283],
        [0.5000, 0.5944, 0.5570, 0.4329],
        [0.5066, 0.5874, 0.5559, 0.4425],
        [0.5056, 0.5884, 0.5560, 0.4411],
        [0.5152, 0.5780, 0.5534, 0.4531],
        [0.5200, 0.5728, 0.5517, 0.4588],
        [0.4954, 0.5994, 0.5574, 0.4251],
        [0.5000, 0.5944, 0.5570, 0.4329],
        [0.4916, 0.6034, 0.5576, 0

In [48]:
prediction_two_stage.round()

tensor([[1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [0., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [0., 1., 1., 0.],
        [0., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [0., 1., 1., 0.],
        [1., 1., 1., 0.],
        [0., 1., 1., 0.],
        [1., 1., 1., 0.],
        [0., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [0., 1., 1., 0.],
        [1., 1., 1., 0.],
        [0., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.]], grad_fn=<RoundBackward0>)

In [51]:
error_2s = prediction_two_stage.round()-pp
error_2s.sum()

tensor(53., grad_fn=<SumBackward0>)

In [52]:
x_two_stage

tensor([0.0676, 0.1042, 0.0000, 0.0782, 0.1407, 0.1115, 0.1261, 0.0202, 0.0334,
        0.1188, 0.0940, 0.0000, 0.1444, 0.0465, 0.0835, 0.0334, 0.0518, 0.0889,
        0.0835, 0.1261, 0.1444, 0.0202, 0.0518, 0.0000, 0.1739, 0.0570],
       grad_fn=<ContinuousOptimizerBackward>)

In [53]:
val_2s, pred_2s, x_2s = eval_opt(net_two_stage, test, test_size)

In [54]:
print(val_2s, pred_2s, x_2s)

0.5218491554260254 tensor([[0.5028, 0.5914, 0.5565, 0.4370],
        [0.5095, 0.5842, 0.5555, 0.4463],
        [0.4734, 0.6227, 0.5620, 0.3868],
        [0.5047, 0.5894, 0.5562, 0.4397],
        [0.5190, 0.5738, 0.5520, 0.4577],
        [0.5114, 0.5821, 0.5548, 0.4486],
        [0.5152, 0.5780, 0.5534, 0.4531],
        [0.4954, 0.5994, 0.5574, 0.4251],
        [0.4972, 0.5974, 0.5573, 0.4283],
        [0.5133, 0.5801, 0.5541, 0.4509],
        [0.5076, 0.5863, 0.5557, 0.4438],
        [0.4916, 0.6034, 0.5576, 0.4186],
        [0.5200, 0.5728, 0.5517, 0.4588],
        [0.4991, 0.5954, 0.5572, 0.4315],
        [0.5056, 0.5884, 0.5560, 0.4411],
        [0.4972, 0.5974, 0.5573, 0.4283],
        [0.5000, 0.5944, 0.5570, 0.4329],
        [0.5066, 0.5874, 0.5559, 0.4425],
        [0.5056, 0.5884, 0.5560, 0.4411],
        [0.5152, 0.5780, 0.5534, 0.4531],
        [0.5200, 0.5728, 0.5517, 0.4588],
        [0.4954, 0.5994, 0.5574, 0.4251],
        [0.5000, 0.5944, 0.5570, 0.4329],
        [0.4916

In [56]:
n = len(ww)
total_2s = 0
for i in range(n):
  p_fail = 1 - x_2s.detach().numpy()*pp.detach().numpy()[:, i]
  p_all_fail = np.prod(p_fail)
  total_2s += ww[i] * (1 - p_all_fail)

In [57]:
total_2s

0.5218493204939957

In [55]:
x_2s.topk(2)

torch.return_types.topk(
values=tensor([0.1739, 0.1444], grad_fn=<TopkBackward0>),
indices=tensor([24, 12]))

In [78]:
[f_true[0][0][0][24], f_true[0][0][0][12]]

[tensor([0, 1, 1, 0]), tensor([0, 0, 0, 0])]

With this model, I would recommend circuits 24 and 12 and I would only get two customers out of a possible optimal four.

## Run Optimizer on Ground Truth

In [80]:
x_ground = ContinuousOptimizer.apply(pp, optfunc, dgrad, None, 0.95)

In [82]:
x_ground

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.5250, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5250, 0.9500, 0.0000])

In [83]:
x_ground.round()

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 1., 0.])

In [84]:
x_ground.topk(2)

torch.return_types.topk(
values=tensor([0.9500, 0.5250]),
indices=tensor([24, 23]))

Running the optimizer on the ground truth set returns one of two optimal solutions. 

This has two implications overall. Improving the model learning algorithm to get closer to the ground truth can lead to an optimal decision. Or decision-focused learning can improve the final decision. 